# Load the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel("data.xlsx")
df

# Remove an empty 'Unnamed: 5' column

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['Unnamed: 5'])
df

# Problem 1:
The problem is as follows: 1) Вычислите общую выручку за июль 2021 по тем сделкам, приход денежных средств, которых не просрочен.

Due to specific construction of a dataset and the fact that we can't simply check for specific datatime value, we'll need to split data on df['status'] == "Июль 2021" and df['status'] == "Август 2021" to create a new dataframe which will include all entries of that specific month

In [ ]:
july_idx = df.index[df['status'] == "Июль 2021"][0]
august_idx = df.index[df['status'] == "Август 2021"][0]

In [ ]:
df_july = df.iloc[july_idx + 1 : august_idx]

Next we'll filter out all rows with ['status'] == "ПРОСРОЧЕНО" and sum everything that's left in column ['sum']

In [ ]:
df_july_valid = df_july[df_july['status'] != "ПРОСРОЧЕНО"]
total_revenue = df_july_valid['sum'].sum()
print(f"Total revenue for July 2021 (not overdue):, {total_revenue:.2f}")

# Problem 2:
Is as follows: Как изменялась выручка компании за рассматриваемый период? Проиллюстрируйте графиком.

There are multiple ways to consider the empty rows for each month, but I've chosen to create a new ['month'] column which will be forward filled with the value of the month 

In [ ]:
# Assuming marker rows (with month names like "Июль 2021") have no revenue value (NaN in 'sum')
df['month'] = np.where(df['sum'].isna(), df['status'], None)
df['month'] = df['month'].ffill()
df

Now let's remove marker rows. We know that all marker rows have NaN in sum column (as shown below)

In [ ]:
df.isnull().sum()

Therefore will use df.dropna on a subset of sum to remove now redundant marker rows

In [ ]:
df_deals = df.dropna(subset=['sum'])
df_deals

Great. Now all that's left is to group everything by month and plot a graph

In [ ]:
monthly_revenue = df_deals.groupby('month')['sum'].sum()
monthly_revenue

In [ ]:
month_order = ["Май 2021", "Июнь 2021", "Июль 2021", "Август 2021", "Сентябрь 2021", "Октябрь 2021"]
monthly_revenue = monthly_revenue.loc[month_order]
plt.plot(month_order, monthly_revenue.values, marker='o')
plt.xlabel('Месяц')
plt.ylabel('Общая выручка (млн.)')
plt.title('Изменение выручки компании за рассматриваемый период')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Problem 3:
The problem is as follows: Кто из менеджеров привлек для компании больше всего денежных средств в сентябре 2021?

For that we'll now use a previously created ['month'] column to get all "Сентябрь 2021" rows

In [ ]:
df_sept = df[df['month'] == "Сентябрь 2021"]

Now let's group rows by the ['sale'] column and sum the revenue in ['sum'] column

In [ ]:
manager_revenue = df_sept.groupby('sale')['sum'].sum()
manager_revenue

In [ ]:
top_manager = manager_revenue.idxmax()
print("Manager with highest revenue in September 2021:", top_manager)

In [ ]:
top_revenue = manager_revenue.max()
print("Total revenue by", top_manager, ":", top_revenue)

# Problem 4:
The problem is as follows: Какой тип сделок (новая/текущая) был преобладающим в октябре 2021?

Again, we'll group by month, but now we'll use value_counts() on a ['new/current'] column

In [ ]:
df_oct = df[df['month'] == "Октябрь 2021"]
deal_type_counts = df_oct['new/current'].value_counts()
print("Количество сделок по типу в октябре 2021:")
print(deal_type_counts)

A simple check on whether or not the deal_type_counts is empty

In [ ]:
if not deal_type_counts.empty:
    predominant_deal_type = deal_type_counts.idxmax()
    print("Преобладающий тип сделок в октябре 2021:", predominant_deal_type)
else:
    print("В октябре 2021 нет данных по сделкам.")

# Problem 5:
Сколько оригиналов договора по майским сделкам было получено в июне 2021?

For that we'll transform ['receiving_date'] column to datetime

In [ ]:
df['receiving_date'] = pd.to_datetime(df['receiving_date'], format='%d.%m.%Y', errors='coerce')

Now we'll use df_jun to store all rows of month June and we'll filter them on ['receiving_date'] column, where values of month == 5 (May), afterwards we'll print out len of the result

In [ ]:
df_jun = df[df['month'] == "Июнь 2021"]
may_originals = df_jun[df_jun['receiving_date'].dt.month == 5]
print(len(may_originals))

# Task 1:
За новые сделки менеджер получает 7 % от суммы, при условии, что статус оплаты «ОПЛАЧЕНО», а также имеется оригинал подписанного договора с клиентом (в рассматриваемом месяце).

We'll add a new column ['month_num'] for ease of filtering later on

In [ ]:
month_mapping = {
    'Январь': 1,
    'Февраль': 2,
    'Март': 3,
    'Апрель': 4,
    'Май': 5,
    'Июнь': 6,
    'Июль': 7,
    'Август': 8,
    'Сентябрь': 9,
    'Октябрь': 10,
    'Ноябрь': 11,
    'Декабрь': 12
}

df['month_num'] = df['month'].apply(lambda x: month_mapping[x.split()[0]] if isinstance(x, str) and x.split() else None)

Now we'll chain filters based on the requirements

In [ ]:
df_new_deals = df[(df['new/current'] == "новая") & (df['status'] == "ОПЛАЧЕНО") & (df['document'] == 'оригинал') & (df['receiving_date'].dt.month == df['month_num'])]
df_new_deals #No new deals that were completed in the same month

Despite the fact that there are no such deals, the logic would be as follows

In [ ]:
commission_by_manager = df_new_deals.groupby('sale')['sum'].sum() * 0.07
print(commission_by_manager)

# Task 2:
За текущие сделки менеджер получает 5 % от суммы, если она больше 10 тыс., и 3 % от суммы, если меньше. При этом статус оплаты может быть любым, кроме «ПРОСРОЧЕНО», а также необходимо наличие оригинала подписанного договора с клиентом (в рассматриваемом месяце).

We'll start by filtering deals by 'current' type and removing all ['status'] == "ПРОСРОЧЕНО"

In [ ]:
df_current = df[(df['new/current'] == "текущая") & (df['status'] != "ПРОСРОЧЕНО") & (df['document'] == "оригинал")].copy() 
df_current

Filter out deals where the original contract was not received in the same month.

In [ ]:
df_pending = df_current[df_current['receiving_date'].dt.month != df_current['month_num']].copy()
df_pending

For each pending deal, determine the bonus rate: 5% if the deal sum > 10,000, otherwise 3%.

In [ ]:
df_pending['bonus_rate'] = df_pending['sum'].apply(lambda x: 0.05 if x > 10000 else 0.03)
df_pending

Calculate the bonus amount for each deal

In [ ]:
df_pending['bonus_amount'] = df_pending['sum'] * df_pending['bonus_rate']
df_pending

Now, group by manager

In [ ]:
balance_by_manager = df_pending.groupby('sale')['bonus_amount'].sum()
balance_by_manager

# Now for the "Вычислите остаток каждого из менеджеров на 01.07.2021" part

We'll start with considering only deals from months prior to July 2021

In [ ]:
df_prior = df_current[df_current['month_num'] < 7].copy()
df_prior

Identify deals that are pending bonus as of 01.07.2021. A deal is pending if the original was not received in its own month. And moreover, if original was received - it must be after or on 01.07.2021
We'll create a mask (True/False values) to filter our df further

In [ ]:
pending_mask = (
    df_prior['receiving_date'].isna() | 
    (df_prior['receiving_date'] >= pd.to_datetime('2021-07-01'))
)

In [ ]:
df_pending = df_prior[pending_mask].copy()
df_pending

Calculate bonus rate and bonus amount per deal (again, since it's a different df)

In [ ]:
df_pending['bonus_rate'] = df_pending['sum'].apply(lambda x: 0.05 if x > 10000 else 0.03)
df_pending['pending_bonus'] = df_pending['sum'] * df_pending['bonus_rate']
df_pending

Group by manager

In [ ]:
pending_balance_by_manager = df_pending.groupby('sale')['pending_bonus'].sum()
print(f"Остаток по бонусам (текущим сделкам) каждого менеджера на 01.07.2021: {pending_balance_by_manager}")